# A First Glimpse At IBM's Quantum Open Science Price Challenge

This post appeared first on [Medium](https://towardsdatascience.com/a-first-glimpse-at-ibms-quantum-open-science-price-challenge-de4a2f41987e) and my weekly [email-course](https://pyqml.substack.com/p/a-first-glimpse-at-ibms-quantum-open?s=w).

IBM just announced its second [Quantum Open Science Prize](https://research.ibm.com/blog/quantum-open-science-prize). They ask for a solution to a quantum simulation problem. They want us to simulate a Heisenberg model Hamiltonian for a three-particle system on IBM Quantum’s 7-qubit Jakarta system using Trotterization.

Even though IBM explains what a Heisenberg model Hamiltonian and Trotterization are, it all appears mysterious, unless you are a physicist or a quantum computing senior.

Fortunately, they also provide a working code example of what they expect. So, let’s see what we computer scientists can learn about the challenge.

First, you need a working Jupyter environment (see my previous post). Second, download the Jupyter notebook (source). We are working on a copy of it because we will make some changes. 

So, let’s go through the code very briefly. First, I stripped away anything unnecessary, such as the classical explanation in section 1.

So, we start with some imports. In line 7, I added the Aer package.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})  # enlarge matplotlib fonts

# Import qubit states Zero (|0>) and One (|1>), and Pauli operators (X, Y, Z)
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit import QuantumCircuit, QuantumRegister, IBMQ, execute, transpile, Aer
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Parameter

# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

/tmp/ipykernel_109/2332860251.py:13: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter


The original code tries to connect to your IBM account and the Jakarta backend in the next step. We skip this and continue directly with the following few cells defining the custom trotterization gate. It is worth noticing that these cells are surrounded with the comments “YOUR TROTTERIZATION GOES HERE — START (beginning of example)” and “FINISH (end of example).”

Apparently, IBM wants us to replace this part with our solution. But, for now, let’s stick to their example.

In [9]:
# YOUR TROTTERIZATION GOES HERE -- START (beginning of example)

# Parameterize variable t to be evaluated at t=pi later
t = Parameter('t')

# Build a subcircuit for XX(t) two-qubit gate
XX_qr = QuantumRegister(2)
XX_qc = QuantumCircuit(XX_qr, name='XX')

XX_qc.ry(np.pi/2,[0,1])
XX_qc.cnot(0,1)
XX_qc.rz(2 * t, 1)
XX_qc.cnot(0,1)
XX_qc.ry(-np.pi/2,[0,1])

# Convert custom quantum circuit into a gate
XX = XX_qc.to_instruction()

# Build a subcircuit for YY(t) two-qubit gate
YY_qr = QuantumRegister(2)
YY_qc = QuantumCircuit(YY_qr, name='YY')

YY_qc.rx(np.pi/2,[0,1])
YY_qc.cnot(0,1)
YY_qc.rz(2 * t, 1)
YY_qc.cnot(0,1)
YY_qc.rx(-np.pi/2,[0,1])

# Convert custom quantum circuit into a gate
YY = YY_qc.to_instruction()

# Build a subcircuit for ZZ(t) two-qubit gate
ZZ_qr = QuantumRegister(2)
ZZ_qc = QuantumCircuit(ZZ_qr, name='ZZ')

ZZ_qc.cnot(0,1)
ZZ_qc.rz(2 * t, 1)
ZZ_qc.cnot(0,1)

# Convert custom quantum circuit into a gate
ZZ = ZZ_qc.to_instruction()

# Combine subcircuits into a single multiqubit gate representing a single trotter step
num_qubits = 3

Trot_qr = QuantumRegister(num_qubits)
Trot_qc = QuantumCircuit(Trot_qr, name='Trot')

for i in range(0, num_qubits - 1):
    Trot_qc.append(ZZ, [Trot_qr[i], Trot_qr[i+1]])
    Trot_qc.append(YY, [Trot_qr[i], Trot_qr[i+1]])
    Trot_qc.append(XX, [Trot_qr[i], Trot_qr[i+1]])

# Convert custom quantum circuit into a gate
Trot_gate = Trot_qc.to_instruction()
XX_qc.draw()
# YOUR TROTTERIZATION GOES HERE -- FINISH (end of example)

┌─────────┐                     ┌──────────┐
q24_0: ┤ Ry(π/2) ├──■───────────────■──┤ Ry(-π/2) ├
       ├─────────┤┌─┴─┐┌─────────┐┌─┴─┐├──────────┤
q24_1: ┤ Ry(π/2) ├┤ X ├┤ Rz(2*t) ├┤ X ├┤ Ry(-π/2) ├
       └─────────┘└───┘└─────────┘└───┘└──────────┘

We don’t yet look into the details of the trotterization, but we continue with the quantum circuit. In the following code, we define the overall quantum circuit (qc) and generate the state tomography circuits to evaluate the fidelity of the simulation. Fidelity is the performance score that we aim to optimize. Fidelity ranges from 0.0 to 1.0, with 1.0 being the best result we could achieve.

In [3]:
# The final time of the state evolution
target_time = np.pi

# Number of trotter steps
trotter_steps = 4  ### CAN BE >= 4

# Initialize quantum circuit for 3 qubits
qr = QuantumRegister(7)
qc = QuantumCircuit(qr)

# Prepare initial state (remember we are only evolving 3 of the 7 qubits on jakarta qubits (q_5, q_3, q_1) corresponding to the state |110>)
qc.x([3,5])  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)

# Simulate time evolution under H_heis3 Hamiltonian
for _ in range(trotter_steps):
    qc.append(Trot_gate, [qr[1], qr[3], qr[5]])

# Evaluate simulation at target_time (t=pi) meaning each trotter step evolves pi/trotter_steps in time
qc = qc.bind_parameters({t: target_time/trotter_steps})

# Generate state tomography circuits to evaluate fidelity of simulation
st_qcs = state_tomography_circuits(qc, [qr[1], qr[3], qr[5]])

# Display circuit for confirmation
# st_qcs[-1].decompose().draw()  # view decomposition of trotter gates
st_qcs[-1].draw()  # only view trotter gates

░          
q4_0: ──────────────────────────────────────────────────────────────░──────────
           ┌────────────┐┌────────────┐┌────────────┐┌────────────┐ ░ ┌─┐      
q4_1: ─────┤0           ├┤0           ├┤0           ├┤0           ├─░─┤M├──────
           │            ││            ││            ││            │ ░ └╥┘      
q4_2: ─────┤            ├┤            ├┤            ├┤            ├─░──╫───────
      ┌───┐│            ││            ││            ││            │ ░  ║ ┌─┐   
q4_3: ┤ X ├┤1 Trot(π/4) ├┤1 Trot(π/4) ├┤1 Trot(π/4) ├┤1 Trot(π/4) ├─░──╫─┤M├───
      └───┘│            ││            ││            ││            │ ░  ║ └╥┘   
q4_4: ─────┤            ├┤            ├┤            ├┤            ├─░──╫──╫────
      ┌───┐│            ││            ││            ││            │ ░  ║  ║ ┌─┐
q4_5: ┤ X ├┤2           ├┤2           ├┤2           ├┤2           ├─░──╫──╫─┤M├
      └───┘└────────────┘└────────────┘└────────────┘└────────────┘ ░  ║  ║ └╥┘
q4_6: ──────────────────────────────────────────────────────────────░──╫──╫──╫─
                                                                    ░  ║  ║  ║ 
c0: 3/═════════════════════════════════════════════════════════════════╩══╩══╩═
                                                                       0  1  2

Finally, the above code displays a drawing of our circuit.

![](./assets/circuit.png)

Our trotterization gate appears four times in the figure. This is what we define in line 5 — the trotter_steps. The comment says that it can be greater than four. Let's keep this in mind because it is one parameter we can work with.

We’re almost ready to execute the circuit. We only need to make up for not setting up the backend earlier. So, we use the Aer package that lets us choose a local simulation backend. Specifically, we use the qasm_simulator—a noiseless simulation backend.


In [4]:
shots = 8192
reps = 8

# WE USE A NOISELESS SIMULATION HERE
backend = Aer.get_backend('qasm_simulator')

jobs = []
for _ in range(reps):
    # execute
    job = execute(st_qcs, backend, shots=shots)
    print('Job ID', job.job_id())
    jobs.append(job)

Job ID 14d0a847-0dbd-4905-a021-4b779ff95fb2
Job ID 0131f678-e28e-413a-85e4-2416c887ae10
Job ID 51d8e139-d3a3-4570-9322-faa2c926a424
Job ID f61efa20-972b-4842-a82a-434a3f45df41
Job ID 643743c7-8ad6-45ef-8b12-44168b9d2510
Job ID 855539c9-e9a8-4e94-bded-83f43e53bdd5
Job ID 119fef4a-f530-497a-91ec-f26d81a415f0
Job ID b4e0c8c9-20d8-4331-aafd-072e1c666aba


We see that we executed the job eight times — the number of replications.

Finally, let’s evaluate the circuit.

In [5]:
# Compute the state tomography based on the st_qcs quantum circuits and the results from those ciricuits
def state_tomo(result, st_qcs):
    # The expected final state; necessary to determine state tomography fidelity
    target_state = (One^One^Zero).to_matrix()  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)
    # Fit state tomography results
    tomo_fitter = StateTomographyFitter(result, st_qcs)
    rho_fit = tomo_fitter.fit(method='lstsq')
    # Compute fidelity
    fid = state_fidelity(rho_fit, target_state)
    return fid

# Compute tomography fidelities for each repetition
fids = []
for job in jobs:
    fid = state_tomo(job.result(), st_qcs)
    fids.append(fid)
    
print('state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean(fids), np.std(fids)))

state tomography fidelity = 0.0003 ± 0.0002


We see a devastating state tomography fidelity of almost 0.

So, let’s see what we can do easily. Since we will run and execute the code a few times, let’s write a wrapper function. The function run_and_evaluate_with_steps takes the number of trotterization steps and the backend to run on as arguments.

In [9]:
def run_and_evaluate_with_steps(steps, backend):
    
    # The final time of the state evolution
    target_time = np.pi

    # Number of trotter steps
    trotter_steps = steps  ### CAN BE >= 4

    # Initialize quantum circuit for 3 qubits
    qr = QuantumRegister(7)
    qc = QuantumCircuit(qr)

    # Prepare initial state (remember we are only evolving 3 of the 7 qubits on jakarta qubits (q_5, q_3, q_1) corresponding to the state |110>)
    qc.x([3,5])  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)

    # Simulate time evolution under H_heis3 Hamiltonian
    for _ in range(trotter_steps):
        qc.append(Trot_gate, [qr[1], qr[3], qr[5]])

    # Evaluate simulation at target_time (t=pi) meaning each trotter step evolves pi/trotter_steps in time
    qc = qc.bind_parameters({t: target_time/trotter_steps})

    # Generate state tomography circuits to evaluate fidelity of simulation
    st_qcs = state_tomography_circuits(qc, [qr[1], qr[3], qr[5]])

    shots = 8192
    reps = 8

    jobs = []
    for _ in range(reps):
        # execute
        job = execute(st_qcs, backend, shots=shots)
        print('Job ID', job.job_id())
        jobs.append(job)

    
    # Compute tomography fidelities for each repetition
    fids = []
    for job in jobs:
        fid = state_tomo(job.result(), st_qcs)
        fids.append(fid)

    print('state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean(fids), np.std(fids)))
        

We can now run the whole code with different numbers of trotter steps. Let’s try it with eight steps.

In [7]:
run_and_evaluate_with_steps(8, Aer.get_backend('qasm_simulator'))

Job ID fe12d5d8-e8d1-4a27-9b3a-48a1792bb12c
Job ID 8202f5b0-02d6-4f1a-bc5b-95260a5721d7
Job ID c0c9e818-8e46-47a6-816b-b054c38daa23
Job ID 50e94f23-9565-430e-9fd8-444fc5b9138c
Job ID 17e7bede-9b28-4b5b-ade6-dea335dabf5a
Job ID ebfd4357-d55a-4a2c-9602-31a9313b5311
Job ID 0f611e73-6f76-416a-b80f-4aca95af9a42
Job ID 2035535f-083c-49c6-b99a-9778fdc34c2f
state tomography fidelity = 0.8543 ± 0.0013


The fidelity is not too bad, is it? So, if increasing the number of trotter steps has such a significant effect, why don’t we try it with even more steps?

In [10]:
run_and_evaluate_with_steps(12, Aer.get_backend('qasm_simulator'))

Job ID 9e8a1c67-0170-4f21-8091-bcb44cdc0d12
Job ID 21dec0ba-d576-487b-b7b9-df6b58aa3bc8
Job ID 4c655b0e-7f8e-4b0c-abc6-770e5886fd72
Job ID e412111a-f44c-49dc-afcf-0cc8fcaac673
Job ID 0678e767-8143-46c6-a6fd-ab277ad1898b
Job ID 7fc47c08-9d90-4793-99eb-e9be1a9c19be
Job ID 296be5dd-b223-433d-bc79-48900f9fafd1
Job ID f51dd663-e76d-4a28-8af1-73eecbc0417e
state tomography fidelity = 0.9687 ± 0.0006


With twelve trotter steps, the fidelity is almost 0.97. That’s practically perfect. So, the exemplary code seems to work pretty well. So, where’s the problem?

IBM asks us to run the circuit on their Jakarta device. Thus far, we simulated a perfect quantum computer. But we know that actual quantum computers are noisy and error-prone. Therefore, let’s simulate an actual device. Qiskit provides the test.mock package that offers simulators corresponding to the behavior of the real devices, such as FakeJakarta.

In [18]:
from qiskit.test.mock import FakeJakarta
device = FakeJakarta()
run_and_evaluate_with_steps(12, device)

Job ID 24465331-3d9c-4388-b7da-e6fe94ed35fe
Job ID 1ef9e74c-ed00-449f-bcb7-cd1c828925c9
Job ID 9f6e23b1-db4b-46db-865c-6b934b476250
Job ID 01234a31-3473-49af-b9d8-6b983ff18316
Job ID ac352d7c-24df-498f-904a-e86e58636c09
Job ID 080c7c2d-daa3-4290-af7f-b21f18b873de
Job ID f4046313-6df5-4f52-869b-2423db698641
Job ID d2da77dd-48cf-4265-8253-7dd62f8a86cc
state tomography fidelity = 0.1442 ± 0.0021


When we run the code with twelve trotter steps on a noisy simulation of the Jakarta device, the fidelity drops to 0.14.

You may want to play with the number of trotter steps. But, you won’t get any fidelity better than 0.2.


So, the actual problem is not to simulate a Heisenberg model Hamiltonian for a three-particle system using Trotterization. Instead, the problem is to do this on a real 7-qubit device.
When we look at the exemplary trotterization gate, we can see that it uses only three qubits. The other four qubits remain unused. The actual challenge is to use these additional four qubits to immunize the circuit against noise and errors.